# Object oriented programming

Imports for the lecture

In [1]:
from typing import List, Any, Iterable
from dataclasses import dataclass
import warnings

# Definitions

*Class and instance*: Classes provide a means of bundling data and functionality together. Creating a new class creates a new _type_ of object, allowing new instances of that type to be made. Each class instance can have attributes attached to it for maintaining its state. Class instances can also have methods (defined by its class) for modifying its state. (https://docs.python.org/3/tutorial/classes.html)

*Constructor*: The constructor is called upon creating the new intance of a class. Python first calls the __new__() method, which is the constructor, to create the object and then calls the __init__() method to initialize the object’s attributes.

## Defining classes

- `class` keyword followed by the name of the class. The Naming convention is CamelCase for classes.
- Instance explicitly bound to the first parameter of each method. One can access the object's attributes using this parameter.
  - Named `self` by convention, so attributes can be accessed like `self.attribute` and methods like `self.method(params)`
- `__init__` is called after the instance is created
  - Not exactly a constructor (that is `__new__`, which is called before `__init__` upon creation) because the instance already exists
  - Not mandatory

In [2]:
class ClassWithInit:
    def __init__(self):
        pass
    
class ClassWithoutInit:
    pass

### Attributes

Attributes are created upon assignment which can happen anywhere not just in `__init__`

In [3]:
class A:
    def __init__(self):
        self.attr1 = 42
        
    def method(self):
        self.attr2 = 43
        
a = A()
print(a.attr1)
# print(a.attr2)  # raises AttributeError, because attr2 does not exist at this point
a.method()
print(a.attr2)

42
43


Attributes can be added to instances:

In [4]:
a.attr3 = 11
print(a.attr3)

11


This will not affect other instances (`__dict__` can be used to access the namespace):

In [5]:
a2 = A()
#a2.attr3  # raises AttributeError
a2.__dict__, a.__dict__

({'attr1': 42}, {'attr1': 42, 'attr2': 43, 'attr3': 11})

By default attributes can be modified outside of the class's functions (they are public).

In [6]:
class B:
    def __init__(self, num: int):
        self.num = num

b = B(2)
print(b.num)
b.num = 3
print(b.num)

2
3


#### Data hiding with name mangling

- Private attributes can be defined through name mangling.
  - Every attribute with at least two leading underscores and at most one trailing underscore is replaced with a mangled attribute.
  - Emulates private behavior.
  - Mangled name: `__classname_attrname`.
- Attributes with one leading underscore should be treated as private.

In [7]:
class A:
    def __init__(self):
        self.__private_attr = 42
        # This should be treated as a private variable.
        self._private = 12
        
    def foo(self):
        self.__private_attr += 1
        
a = A()
a.foo()
# a.__private_attr  # raises AttributeError
# a._private = 13  # Considered private, but won't raise errors
a.__dict__

{'_A__private_attr': 43, '_private': 12}

## Special attributes

- Every object has a number of special attributes.
- They follow the **double underscore or dunder** notation: `__attribute__`.
- Most of them are automatically created.
- Advanced OOP features are implemented using these.
- **dunder** should be avoided when naming an attribute.


**`__dict__`**: implements a class' (module's) namespace:

In [8]:
A.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.A.__init__(self)>,
              'foo': <function __main__.A.foo(self)>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [9]:
a2.__dict__

{'attr1': 42}

## Class attributes

- Class attributes are class-global attributes.
- By default an attribute is object level.

In [10]:
class A:
    class_attr = 42

Accessing class attributes via instances:

In [11]:
a1 = A()
print(a1.class_attr)

42


Accessing class attributes via the class object:

In [12]:
print(A.class_attr)

42


Setting the class object via the class:

In [13]:
a1 = A()
a2 = A()

print(a1.class_attr, a2.class_attr)
A.class_attr = 43
a1.class_attr, a2.class_attr

42 42


(43, 43)

They cannot be set via an instance:

In [14]:
a1 = A()
a2 = A()
a1.class_attr = 11
print(a1.class_attr)
print(a2.class_attr)

11
43


Because this assignment creates a new attribute in the *instance's namespace*.

In [15]:
a1.__dict__

{'class_attr': 11}

In [16]:
a2.__dict__

{}

In [17]:
a2.__class__

__main__.A

In [18]:
# A.__dict__
a2.__class__.__dict__

mappingproxy({'__module__': '__main__',
              'class_attr': 43,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

## Methods

Functions inside the class definition.

They explicitly take the instance as first parameter named `self` by convention. The previously mentioned `__init__` and `__new__` are also methods of the given class.

In [19]:
class A:
    def foo(self):
        print("foo called")
    
    def bar(self, param: Any):
        print("bar called with parameter {}".format(param))
        
a = A()
a.foo()
a.bar(42)

foo called
bar called with parameter 42


__Calling methods__

1. `instance.method(param)`
2. `class.method(instance, param)`

In [20]:
c = A()
c.foo()
c.bar(42)
A.foo(c)
A.bar(c, 43)

foo called
bar called with parameter 42
foo called
bar called with parameter 43


# Class decorators

Many OO features are achieved via a syntax sugar called decorators. We will talk about decorators in detail later.

The most common features are:

1. staticmethod,
1. classmethod,
1. property.

## Static method

Static methods are not bound to an instance of the class. You can define it using the `@staticmethod` decorator (see next lecture for more information).

You don't give the `self` parameter to these methods.

In [21]:
class B:
    @staticmethod
    def print_list(iterable_object: Iterable):
        for elem in iterable_object:
            print(elem)

You can call this method on an instance as well as on the class.

In [22]:
iterable_list = ["apple", "banana", "cherry", "date"]

b = B()

print("Intance\n")
b.print_list(iterable_list)
print("\nClass\n")
B.print_list(iterable_list)

Intance

apple
banana
cherry
date

Class

apple
banana
cherry
date


## Class method

Typically used as factory methods for the class. Bound to the class instead of an instance of the class, defined using the `@classmethod` decorator.

Its first argument is a class instance called `cls` by convention.

In [23]:
class Complex(object):
    def __init__(self, real: float, imag: float):
        self.real = real
        self.imag = imag
        
    def __str__(self):
        return f"{self.real}+j{self.imag}"
    
    @classmethod
    def from_str(cls, complex_str: str):
        print(f"{cls.__name__}.from_str")
        real, imag = complex_str.split('+')
        imag = imag.lstrip('ij')
        return cls(float(real), float(imag))
    
class SuperComplex(Complex):
    pass
    
c1 = Complex.from_str("3.45+j2")
print(c1)
c2 = Complex(3, 4)
print(c2)

s = SuperComplex.from_str("3+i0")
print(s)
type(s), type(c1)

Complex.from_str
3.45+j2.0
3+j4
SuperComplex.from_str
3.0+j0.0


(__main__.SuperComplex, __main__.Complex)

*What's the difference between static method and class method?* The classmethod has acces to the classes inner state, so it can use and modify it.

## Properties

Properties are attributes with getters, setters and deleters, defined with the `@property` decorator. Property works as both a built-in function and as separate decorators. Usefull to protect attributes from invalid values.

In [24]:
class Person(object):
    def __init__(self, name: str, age: int):
        self.name = name
        self.age = age
        
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, age: int):
        try:
            if 0 <= age <= 150:
                self._age = age
        except TypeError:
            pass
        
    @age.deleter
    def age(self):
        print("Del")
            
    def __str__(self):
        return f"Name: {self.name}, age: {self.age}"
            

p = Person("John", 12)
print(p)
p.age = "abc"
print(p)
p.age = 85
print(p)
p.age = 850
print(p)
del p.age

Name: John, age: 12
Name: John, age: 12
Name: John, age: 85
Name: John, age: 85
Del


# Reflection

__`getattr`__ gets an attribute by name:


In [25]:
class A:
    def __init__(self, value: Any):
        self.attr1 = value
        
getattr(A(12), 'attr1')

12

Missing attributes raise `AttributeError`:

In [26]:
# getattr(A(12), 'attr2')  # raises AttributeError

A default can be specified:

In [27]:
getattr(A(12), 'attr2', "attr2_default")

'attr2_default'

__`setattr`__ sets an attribute:

In [28]:
class A:
    pass

# attr_name = input("Attribute name = ")
# attr_value = input(f"{attr_name} = ")
attr_name = 'apple'
attr_value = 15

a = A()
setattr(a, attr_name, attr_value)
print(f"{getattr(a, attr_name) = }")
print(f"{a.apple = }")
a.__dict__

getattr(a, attr_name) = 15
a.apple = 15


{'apple': 15}

__`delattr`__ deletes an attribute from the namespace:

In [29]:
class A:
    def __init__(self):
        self.attr1 = 1
        self.attr2 = 2
        
a = A()
print(a.__dict__)

delattr(a, 'attr1')
print(a.__dict__)

{'attr1': 1, 'attr2': 2}
{'attr2': 2}


__`hasattr`__ checks if an attribute is present in the parameter's namespace:

In [30]:
hasattr(a, 'attr1'), hasattr(a, 'attr2')

(False, True)

# Data class

 - It is a new feature in Python 3.10
 - Contains class variables and has a default `__init__` with the attributes as the parameters
 - The syntax of these contains the `@dataclass` decorator
 
(https://docs.python.org/3/library/dataclasses.html)

In [31]:
@dataclass
class A:
    name: str
    values: List[float]

    def add_values(self) -> float:
        return sum(self.values)


# a = A() # Will raise TypeError, as the default init requires name and values passed to is
a = A("dataclass", [0.2, 0.3])
a.add_values()

0.5

You can define if a dataclass implements certain functions by default. For full list see the documentation linked above.
Some examples are below

Init

In [32]:
@dataclass(init=True) # the default
class A:
    name: str

@dataclass(init=False)
class B:
    name: str

a = A("apple")
print(a.name)
b = B()
# print(b.name) # Will raise AttributeError, because B has no 'name' attribute
# b = B("banana") # Will raise TypeError, because the default init does not take arguments

apple


Repr: whether to generate a default, nicely formatted string representation of the instance

In [33]:
@dataclass(repr=True) # the default
class A:
    name: str

@dataclass(repr=False)
class B:
    name: str

a = A("apple")
print(a)
b = B("banana")
print(b)

A(name='apple')


Eq: whether to define a default equal comparator function based on the tuple of the attributes

In [34]:
@dataclass(eq=True) # the default
class A:
    name: str
    isinteresing: bool


@dataclass(eq=False)
class B:
    name: str
    isinteresing: bool

a = A("apple", True)
b = B("banana", True)

c = A("apple", True)
d = B("banana", True)
print(a == c)
print(b == d)

True
False


# Inheritance

Python supports inheritance and multiple inheritance.

In [35]:
class A:
    pass

class B(A):
    pass

a = A()
b = B()
print(isinstance(a, B))
print(isinstance(b, A))
print(issubclass(B, A))
print(issubclass(A, B))

False
True
True
False


In [36]:
isinstance(3, int), isinstance(3, float)

(True, False)

Python 3 implicitly subclasses `object`:

In [37]:
class A: pass
class B(object): pass

print(issubclass(A, object))
print(issubclass(B, object))

True
True


__Method inheritance__

Methods are inherited and overridden in the usual way:

In [38]:
class A(object):
    def foo(self):
        print("A.foo was called")
        
    def bar(self):
        print("A.bar was called")
        
class B(A):
    def foo(self):
        print("B.foo was called")
        
b = B()
b.foo()
b.bar()

B.foo was called
A.bar was called


Since data attributes can be created anywhere, they are only inherited if the code in the base class' method is called.

In [39]:
class A(object):
    
    def foo(self):
        self.value = 42
        
class B(A):
    pass

b = B()
print(b.__dict__)
a = A()
print(a.__dict__)
a.foo()
print(a.__dict__)

{}
{}
{'value': 42}


__Calling the base class's `init`__

Since `__init__` is not a constructor, the base class' init is not called automatically, if the subclass overrides it

In [40]:
class A(object):
    def __init__(self):
        print("A.__init__ called")
        
class B(A):
    def __init__(self):
        print("B.__init__ called")
        
class C(A):
    pass
        
b = B()
print("Creating c")
c = C()

B.__init__ called
Creating c
A.__init__ called


The base class's methods can be called in at least two ways:
1. Explicitely via the class name
1. Using the **super** function

In [41]:
class A(object):
    def __init__(self):
        print("A.__init__ called")
        
class B(A):
    def __init__(self):
        A.__init__(self)
        print("B.__init__ called")
        
class C(A):
    def __init__(self):
        super().__init__()
        print("C.__init__ called")
        
print("Instantiating B")
b = B()
print("\nInstantiating C")
c = C()

Instantiating B
A.__init__ called
B.__init__ called

Instantiating C
A.__init__ called
C.__init__ called


A complete example:

In [42]:
class Person(object):
    
    def __init__(self, name: str, age: int):
        self.name = name
        self.age = age
        
    def __str__(self):
        return f"{self.name}, age {self.age}"
        
class Employee(Person):
    
    def __init__(self, name: str, age: int, position: str, salary: int):
        self.position = position
        self.salary = salary
        super().__init__(name, age)
        
    def __str__(self):
        return f"{super().__str__()}, position: {self.position}, salary: {self.salary}"
    
    
e = Employee("John Smith", 33, "manager", 450000)
print(e)
print(Person(e.name, e.age))

John Smith, age 33, position: manager, salary: 450000
John Smith, age 33


## Duck typing and interfaces

- No built-in mechanism for interfacing.
- The Abstract Base Classes (`abc`) module implements interface-like features.
- Not used extensively in Python in favor of duck typing.

"In computer programming, duck typing is an application of the duck test in type safety. It requires that type checking be deferred to runtime, and is implemented by means of dynamic typing or reflection." -- [Wikipedia](https://en.wikipedia.org/wiki/Duck_typing)

"If it looks like a duck, swims like a duck, and quacks like a duck, then it probably is a duck." -- [Wikipedia](https://en.wikipedia.org/wiki/Duck_test)

Allows polymorphism without abstract base classes.

In [43]:
class Cat(object):
    
    def make_sound(self):
        self.meow()
        
    def meow(self):
        print("Meow")
        
        
class Dog(object):
    
    def make_sound(self):
        self.bark()
        
    def bark(self):
        print("Woof")
        

animals = [Cat(), Dog()]
for animal in animals:
    # animal must have a make_sound method
    animal.make_sound()

Meow
Woof


__`NotImplementedError`__

In [44]:
class Animal(object):
    def make_sound(self):
        raise NotImplementedError("Animal subclass must implement make_sound.")
        
class Dog(Animal):
    def make_sound(self):
        print("Woof")
        
class Fox(Animal):
    pass
        
dog = Dog()

# Fox can be instantiated
fox = Fox()

dog.make_sound()
# fox.make_sound() # This raises a NotImplementedError, because this method is not overriden in Fox

Woof


__abc module__

More options in the [Abstract Base Classes module](https://docs.python.org/3/library/abc.html).

# Magic methods

Mechanism that allows implementing advanced OO features.

**dunder** methods.

`__str__` __method__

Returns the string representation of the object:

In [45]:
class A(object):
    def __init__(self, value: float=42):
        self.param = value
        
    def __str__(self):
        return f"My id is {id(self)} and my parameter is {self.param}."
    
print(A(345))

My id is 140490244439536 and my parameter is 345.


## Operator overloading

- Operators are mapped to magic functions.
- Defining these functions defines/overrides operators.
- Comprehensive list of operator functions are [here](https://docs.python.org/3/library/operator.html).
- Many built-in functions are included as well:
  - `__len__`: defines the behavior of `len(obj)`
  - `__abs__`: defines the behavior of `abs(obj)`
- Dataclass can generate some of these automatically

In [46]:
class Complex(object):
    def __init__(self, real: float=0.0, imag: float=0.0):
        self.real = real
        self.imag = imag
        
    def __abs__(self):
        return (self.real**2 + self.imag**2) ** 0.5
    
    def __eq__(self, other: Complex):  # right hand side
        return self.real == other.real and self.imag == other.imag
    
    def __gt__(self, other: Complex | int | float):
        return abs(self) > abs(other)
    
    def __add__(self, other: Complex):
        return Complex(self.real+other.real, self.imag+other.imag)
    
    def __repr__(self):
        return f"{self.real}+j{self.imag}"
    
c1 = Complex()
c2 = Complex(1, 1)
print(f"{c1 == c1 = }")
print(f"{c1 > c2 = }")
print(f"{abs(c2) = }")
print(f"{c1 + c2 + c2 = }")

c1 == c1 = True
c1 > c2 = False
abs(c2) = 1.4142135623730951
c1 + c2 + c2 = 2.0+j2.0


## Assignment operator, shallow copy and deep copy


There are 3 types of assignment and copying:

1. the assignment operator (=) creates a new reference to the same object,
1. `copy` performs shallow copy,
1. `deepcopy` recursively deepcopies everything.


__Assignment operator__:
- It **cannot** be overridden.
- It performs name binding instead of copying.
- Tightly bound to the garbage collector.

The difference between shallow and deep copy is only relevant for compound objects (objects that contain other objects).

In [47]:
l1 = [[1, 2], [3, 4, 5]]
l2 = l1
print(f"{(l1 is l2) = }")
print(f"{(l1[0] is l2[0]) = }")

(l1 is l2) = True
(l1[0] is l2[0]) = True


In [48]:
l1[0][0] = 10
l2

[[10, 2], [3, 4, 5]]

__Shallow copy__

In [49]:
from copy import copy

l1 = [[1, 2], [3, 4, 5]]
l2 = copy(l1)

print(f"{(l1 is l2) = }")
print(f"{(l1[0] is l2[0]) = }")
l1[0][0] = 10
l2

(l1 is l2) = False
(l1[0] is l2[0]) = True


[[10, 2], [3, 4, 5]]

In [50]:
l1 = [[1, 2], [3, 4, 5]]
l2 = copy(l1)
print(f"{(l1 is l2) = }")
print(f"{(l1[0] is l2[0]) = }")

(l1 is l2) = False
(l1[0] is l2[0]) = True


In [51]:
l1 == l2

True

In [52]:
l1 = [[1, 2], [3, 4, 5]]
l2 = copy(l1)
l2[0][0] = 11
l1

[[11, 2], [3, 4, 5]]

In [53]:
l1 = [[1, 2], [3, 4, 5]]
l2 = copy(l1)
l1[0] = [7, 8, 9]
print(f"{l1 = }")
print(f"{l2 = }")
print(f"{(l1[0] is l2[0]) = }")
print(f"{(l1[1] is l2[1]) = }")

l1 = [[7, 8, 9], [3, 4, 5]]
l2 = [[1, 2], [3, 4, 5]]
(l1[0] is l2[0]) = False
(l1[1] is l2[1]) = True


__Deep copy__

In [54]:
from copy import deepcopy

l1 = [[1, 2], [3, 4, 5]]
l2 = deepcopy(l1)
print(f"{(l1 is l2) = }")
print(f"{(l1[0] is l2[0]) = }")

(l1 is l2) = False
(l1[0] is l2[0]) = False


In [55]:
l1[0][0] = 10
l2

[[1, 2], [3, 4, 5]]

Both can be defined via magic methods

*note that these implementations do not check for infinite loops*

In [56]:
from copy import copy, deepcopy

class ListOfLists(object):
    def __init__(self, lists):
        self.lists = lists
        self.list_lengths = [len(l) for l in self.lists]
        
    def __copy__(self):
        print("ListOfLists copy called")
        return ListOfLists(copy(self.lists))
        
    def __deepcopy__(self, memo):
        print("ListOfLists deepcopy called")
        return ListOfLists(deepcopy(self.lists))
        
l1 = ListOfLists([[1, 2], [3, 4, 5]])
l2 = copy(l1)
l1.lists[0][0] = 12
print(l2.lists)
l3 = deepcopy(l1)

ListOfLists copy called
[[12, 2], [3, 4, 5]]
ListOfLists deepcopy called


However, these are very far from complete implementations. We need to take care of preventing infinite loops and support for pickling (serialization module).

## Object creation and destruction: the `__new__` and the `__del__` method

The `__new__` method is called to create a new instance of a class. `__new__` is a static method that takes the class object as a first parameter.

Typical implementations create a new instance of the class by invoking the superclass’s `__new__()` method using `super(currentclass, cls).__new__(cls[, ...])` with appropriate arguments and then modifying the newly-created instance as necessary before returning it.

`__new__` has to return an instance of `cls`, on which `__init__` is called.

The `__del__` method is called when an object is about to be destroyed.
Although technically a destructor, it is handled by the garbage collector.
It is not guaranteed that `__del__()` methods are called for objects that still exist when the interpreter exits.

In [57]:
class A(object):
    
    @classmethod
    def __new__(cls, *args, **kwargs):
        instance = super().__new__(cls)
        print("A.__new__ called")
        return instance
    
    def __init__(self):
        print("A.__init__ called")
        
    def __del__(self):
        print("A.__del__ called")
        try:
            super(A, self).__del__()
        except AttributeError:
            print("parent class does not have a __del__ method")
        
        
a = A()
print("a created")
del a

A.__new__ called
A.__init__ called
a created
A.__del__ called
parent class does not have a __del__ method


## Object introspection

- Python has full object introspection.
- **`dir`** lists every attribute of an object

In [58]:
class A(object):
    var = 12
    def __init__(self, value: Any):
        self.value = value
        
    def foo(self):
        print("bar")
  
", ".join(dir(A))

'__class__, __delattr__, __dict__, __dir__, __doc__, __eq__, __format__, __ge__, __getattribute__, __gt__, __hash__, __init__, __init_subclass__, __le__, __lt__, __module__, __ne__, __new__, __reduce__, __reduce_ex__, __repr__, __setattr__, __sizeof__, __str__, __subclasshook__, __weakref__, foo, var'

Class `A` does not have a value attribute, since it is bounded to an instance. However, it does have the class global var attribute.

An instance of A has both:

In [59]:
", ".join(dir(A(12)))

'__class__, __delattr__, __dict__, __dir__, __doc__, __eq__, __format__, __ge__, __getattribute__, __gt__, __hash__, __init__, __init_subclass__, __le__, __lt__, __module__, __ne__, __new__, __reduce__, __reduce_ex__, __repr__, __setattr__, __sizeof__, __str__, __subclasshook__, __weakref__, foo, value, var'

`__code__`

Every object has a `__code__` attribute, which contains everything needed to call the function.

In [60]:
def evaluate(x: Any):
    a = 12
    b = 3
    return a*x + b
    
print(evaluate.__code__)
dir(evaluate.__code__)

<code object evaluate at 0x7fc6948eb5d0, file "/tmp/ipykernel_820565/506748482.py", line 1>


['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'co_argcount',
 'co_cellvars',
 'co_code',
 'co_consts',
 'co_filename',
 'co_firstlineno',
 'co_flags',
 'co_freevars',
 'co_kwonlyargcount',
 'co_lines',
 'co_linetable',
 'co_lnotab',
 'co_name',
 'co_names',
 'co_nlocals',
 'co_posonlyargcount',
 'co_stacksize',
 'co_varnames',
 'replace']

In [61]:
evaluate.__code__.co_varnames, evaluate.__code__.co_freevars, evaluate.__code__.co_stacksize

(('x', 'a', 'b'), (), 2)

The **inspect** module provides further code introspection tools, including the **getsourcelines** function, which returns the source code itself.

In [62]:
from inspect import getsourcelines

print("".join(getsourcelines(evaluate)[0]))

def evaluate(x: Any):
    a = 12
    b = 3
    return a*x + b



# Multiple inheritance

- No interface inheritance in Python.
- Since every class subclasses `object`, the diamond problem is present.
- Method resolution order (MRO) defines the way methods are inherited

In [63]:
class A(object):
    def __init__(self, value):
        print("A init called")
        self.value = value
        
class B(object):
    def __init__(self):
        print("B init called")

class C(A, B):
    def __init__(self, value1, value2):
        print("C init called")
        self.value2 = value2
        super().__init__(value1)
        
class D(A, B): pass
        
print("Instantiating C")
c = C(1, 2)
print("Instantiating D")
d = D(12)

Instantiating C
C init called
A init called
Instantiating D
A init called


# Exception

We have seen examplels already of different type of exceptions during the lecture.
An exception is raised, when the code tries to do something illegal, like dividing by zero:

In [64]:
# a = 8 / 0  # will raise ZeroDivisionError

## Handling exceptions

We can prevent the exception from exiting the code:

In [65]:
try:
    a = 8 / 0
except ZeroDivisionError as e:
    print(f"The code would have raised a {type(e)}")

The code would have raised a <class 'ZeroDivisionError'>


We can even define an additional code block, that will run every time after the try-except block no matter the outcome. These are called *clean-up actions*:

In [66]:
try:
    a = 8 / 0
except ZeroDivisionError as e:
    print(f"The code would have raised a {type(e)}")
    # raise e  # We raise the exception we just caught
finally:
    print("We are in finally")

The code would have raised a <class 'ZeroDivisionError'>
We are in finally


The exception can be a list, a specific type, a general exception or not named at all. If we want to keep the information about the exception we can do so with a context managed variable.

List of exceptions, if the code might raise multiple type of exceptions:

In [67]:
try:
    l = [0, 1, 2]
    a = l[10] / l[0]
except (IndexError, ZeroDivisionError) as e:
    print(f"The code would have raised a {type(e)}")

The code would have raised a <class 'IndexError'>


General:

In [68]:
try:
    a = 8 / 0
except Exception as e:
    print(f"The code would have raised a {type(e)}")

The code would have raised a <class 'ZeroDivisionError'>


Without any named exception class

In [69]:
try:
    a = 8 / 0
except:
    print(f"The code would have raised an exception")

The code would have raised an exception


## Raise

We can also raise exceptions by using the `raise` keyword and creating an exception instance.

In [70]:
try:
    raise ValueError()
except ValueError:
    print("We caught the ValueError!")

We caught the ValueError!


We can also pass attributes to the exception to provide a custom error message.

In [71]:
try:
    a = 12
    raise ValueError(f"The a value is {a}")
except ValueError as e:
    print(f"The error message is: {e}")

The error message is: The a value is 12


We can chain exceptions as well, when catching an exception raises an other.

In [72]:
"""
try:
    raise ValueError("This is the value error")
except ValueError:
    raise AttributeError("This is an AttributeError")
"""

'\ntry:\n    raise ValueError("This is the value error")\nexcept ValueError:\n    raise AttributeError("This is an AttributeError")\n'

## Assert

Asserts are short tests, that raise `AssertionError` if the specified condition is false.

In [73]:
a = 43
b = 42

assert a > b

We can also add an error message to the assertion

In [74]:
# assert a == b, "This is the assertion message"  # Will raise AssertionError

## Warnings

Warnings are special exceptions, that won't stop the running of the code, but will print a message to stderr. You need the warnings module for this, as raising the warning will create an error.

In [75]:
warnings.warn(RuntimeWarning("This is just a warning, no worries"))
# warnings.warn("This is just a warning, no worries", RuntimeWarning)  # same as above, just different syntax
print("We can still reach this code")

We can still reach this code


/tmp/ipykernel_820565/2122194017.py:1: RuntimeWarning: This is just a warning, no worries
  warnings.warn(RuntimeWarning("This is just a warning, no worries"))


We don't need to create a Warning instance to write a warning. In this case the type of the warning will be `UserWarning`

In [76]:
warnings.warn("Warning message")

/tmp/ipykernel_820565/3796036779.py:1: UserWarning: Warning message
  warnings.warn("Warning message")


You cannot use warning for an error.

In [77]:
# warnings.warn("This is an error, I am warning you", ValueError)  # will raise TypeError

## Writing your own exception

Typically an exception is a subclass of the Exception. All exceptions are derived from BaseException.

In [78]:
class MyException(Exception):
    pass

# raise MyException("Works the same as the builtin")

For Warnings use Warning as the super class.

In [79]:
class MyWarning(Warning):
    pass

warnings.warn("This is my warning", MyWarning)

/tmp/ipykernel_820565/2695896824.py:4: MyWarning: This is my warning
  warnings.warn("This is my warning", MyWarning)


# See also

* [Classes (official documentation)](https://docs.python.org/3/tutorial/classes.html)
* [Data model (official documentation)](https://docs.python.org/3/reference/datamodel.html)
* [Data class (official documentation)](https://docs.python.org/3/library/dataclasses.html)
* [Errors and exceptions (official documentation)](https://docs.python.org/3/tutorial/errors.html)